In [1]:
import sys
sys.path.append("E:\\Document\\bitcoin\\trade-executor")

In [9]:
import pandas as pd

In [2]:
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in C:\Users\DESKTOP/.tradingstrategy


c:\Users\DESKTOP\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Check Top Pairs

In [14]:
from tradingstrategy.exchange import ExchangeUniverse
columnar_pair_table = client.fetch_pair_universe()
all_pairs_dataframe = columnar_pair_table.to_pandas()
universe = client.fetch_exchange_universe()

In [19]:
import pandas as pd
from tradingstrategy.chain import ChainId

# Convert PyArrow table to Pandas format to continue working on it
all_pairs_dataframe = columnar_pair_table.to_pandas()

# Filter down to pairs that only trade on Sushiswap
sushi_swap = universe.get_by_chain_and_name(ChainId.ethereum, "sushi")
sushi_pairs: pd.DataFrame = all_pairs_dataframe.loc[all_pairs_dataframe['exchange_id'] == sushi_swap.exchange_id]

print(f"Sushi on Ethereum has {len(sushi_pairs)} pairs")

Sushi on Ethereum has 810 pairs


In [20]:
sushi_pairs = sushi_pairs.copy()
total_volume_30d = sushi_pairs["buy_volume_30d"] + sushi_pairs["sell_volume_30d"]
sushi_pairs["total_30d_volume"] = total_volume_30d
# Sort by highest volume first
sushi_pairs = sushi_pairs.sort_values("total_30d_volume", ascending=False)

print("total_30d_volume column sum added")

total_30d_volume column sum added


In [22]:
output = sushi_pairs[["base_token_symbol", "quote_token_symbol", "total_30d_volume"]].copy()

output.style.format({
  'total_30d_volume': lambda val: f'${val:,.2f}',
})

output = output.rename(columns={
    'base_token_symbol': 'Base token',
    'quote_token_symbol': 'Quote token',
    'total_30d_volume': 'Volume 30d USD'})

top_10 = output.head(10)

top_10.style.format({
  'Volume 30d USD': lambda val: f'${val:,.2f}',
}).hide_index()

C:\Users\DESKTOP\AppData\Local\Temp\ipykernel_31552\4064177701.py:16: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  }).hide_index()


Base token,Quote token,Volume 30d USD
WETH,USDC,"$84,757,200.00"
LUNA,WETH,"$83,835,640.00"
CEL,WETH,"$39,472,288.00"
CVP,WETH,"$37,817,524.00"
WETH,USDT,"$37,034,896.00"
WETH,UST,"$35,645,980.00"
SYN,WETH,"$33,053,244.00"
SUSHI,WETH,"$26,768,100.00"
ILV,WETH,"$22,474,072.00"
RBIS,WETH,"$21,087,714.00"


Maybe it's better to use Sushi-ETH*ETH-USDC as Sushi-USDC data since Sushi-USDC itself is not that liquidated.

## Check Pair ID
Pair id is something in trading strategy like a key to access each pair.

In [23]:
from tradingstrategy.timebucket import TimeBucket
from tradingstrategy.pair import PandasPairUniverse

raw_pairs = client.fetch_pair_universe().to_pandas()
raw_liquidity_samples = client.fetch_all_liquidity_samples(TimeBucket.d1).to_pandas()

pair_universe = PandasPairUniverse(raw_pairs)

In [24]:
sushi_swap = universe.get_by_chain_and_name(ChainId.ethereum, "sushi")

In [25]:
pair_universe.get_one_pair_from_pandas_universe(sushi_swap.exchange_id, "SUSHI", "WETH")

<Pair #9262 SUSHI - WETH (0x795065dcc9f64b5614c407a6efdc400da6221fb0) at exchange #22 on ethereum>

In [64]:
pair_universe.get_one_pair_from_pandas_universe(sushi_swap.exchange_id, "WETH", "USDC")

<Pair #9261 WETH - USDC (0x397ff1542f962076d0bfe58ea045ffa2d347aca0) at exchange #22 on ethereum>

In [27]:
pair_universe.get_one_pair_from_pandas_universe(sushi_swap.exchange_id, "SUSHI", "USDC")

<Pair #9810 SUSHI - USDC (0xba87dc891945dbb3caeeaf822de208d7ea89b298) at exchange #22 on ethereum>

## Download Candles

In [48]:
from tradingstrategy.timebucket import TimeBucket

# Download all 24h candles as Parquet columnar data
all_candles = client.fetch_all_candles(TimeBucket.h1)

In [4]:
import pyarrow.parquet as pq

pq.write_table(all_candles, 'all_candles.parquet')

In [5]:
all_candles_dataframe = all_candles.to_pandas()

In [49]:
sushi_usdc = all_candles_dataframe.loc[all_candles_dataframe['pair_id']==9810]
sushi_usdc['volume'] = sushi_usdc['buy_volume']+sushi_usdc['sell_volume']
sushi_usdc=sushi_usdc.set_index('timestamp')

C:\Users\DESKTOP\AppData\Local\Temp\ipykernel_31552\3305148503.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sushi_usdc['volume'] = sushi_usdc['buy_volume']+sushi_usdc['sell_volume']


In [50]:
eth_usdc = all_candles_dataframe.loc[all_candles_dataframe['pair_id']==9261]
eth_usdc['volume'] = eth_usdc['buy_volume']+eth_usdc['sell_volume']
eth_usdc=eth_usdc.set_index('timestamp')


C:\Users\DESKTOP\AppData\Local\Temp\ipykernel_31552\2589540203.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eth_usdc['volume'] = eth_usdc['buy_volume']+eth_usdc['sell_volume']


In [51]:
sushi_eth = all_candles_dataframe.loc[all_candles_dataframe['pair_id']==9262]
sushi_eth['volume'] = sushi_eth['buy_volume']+sushi_eth['sell_volume']
sushi_eth=sushi_eth.set_index('timestamp')

C:\Users\DESKTOP\AppData\Local\Temp\ipykernel_31552\3619037984.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sushi_eth['volume'] = sushi_eth['buy_volume']+sushi_eth['sell_volume']


In [52]:
sushi_eth_merge = pd.merge(sushi_usdc['close'], eth_usdc['close'], left_index=True, right_index=True, suffixes=['_sushi','_eth'])
print(len(sushi_eth_merge))

1949


In [53]:
sushi_eth_merge = pd.merge(eth_usdc['close'], sushi_eth['close'], left_index=True, right_index=True, suffixes=['_sushi','_eth'])
print(len(sushi_eth_merge))

16831


In [61]:
sushi_eth

,pair_id,exchange_rate,open,close,high,low,buys,sells,buy_volume,sell_volume,avg,start_block,end_block,volume
timestamp,,,,,,,,,,,,,,
2020-09-09 19:00:00,9262,359.647797,3.052969,2.866729,3.053297,2.864931,41,94,3.029540e+05,2.476766e+06,0.008241,10829341,10829571,2.779720e+06
2020-09-09 20:00:00,9262,357.330719,2.866030,2.784903,2.867045,2.774082,33,91,1.067059e+06,1.637232e+06,0.007877,10829576,10829846,2.704291e+06
2020-09-09 21:00:00,9262,355.744598,2.771499,2.781855,2.801418,2.771499,40,54,4.252561e+05,6.672722e+05,0.007827,10829858,10830121,1.092528e+06
2020-09-09 22:00:00,9262,356.252808,2.780357,2.835084,2.848392,2.780357,76,47,1.421558e+06,6.617590e+05,0.007920,10830127,10830376,2.083318e+06
2020-09-09 23:00:00,9262,355.006378,2.835865,2.850358,2.894154,2.820720,43,86,1.820479e+06,1.277800e+06,0.008057,10830382,10830651,3.098280e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-20 04:00:00,9262,1628.919556,1.202151,1.196493,1.202151,1.196493,2,2,4.309409e+03,4.574720e+03,0.000737,15375453,15375537,8.884129e+03
2022-08-20 05:00:00,9262,1639.221558,1.200901,1.201418,1.209376,1.200901,0,4,0.000000e+00,8.319121e+03,0.000734,15375699,15375891,8.319121e+03
2022-08-20 06:00:00,9262,1640.021729,1.198095,1.211688,1.211688,1.198095,2,1,3.200384e+03,5.189347e+01,0.000734,15376056,15376168,3.252278e+03


Better to use Sushi-ETH*ETH-USDC

In [54]:
sushi_eth = sushi_eth.reindex(sushi_eth_merge.index)
eth_usdc = eth_usdc.reindex(sushi_eth_merge.index)


In [62]:
sushi_usdc = pd.DataFrame(columns=sushi_eth.columns, index=sushi_eth.index)
sushi_usdc[['open','close','high','low']] = sushi_eth[['open','close','high','low']]*eth_usdc[['open','close','high','low']]
sushi_usdc['volume'] = sushi_eth['volume']+eth_usdc['volume']
sushi_usdc[['open','close','high','low','volume']].to_csv('sushi_usdc.csv')

In [56]:
eth_usdc[['open','close','high','low','volume']].to_csv('eth_usdc.csv')

In [63]:
sushi_usdc

,pair_id,exchange_rate,open,close,high,low,buys,sells,buy_volume,sell_volume,avg,start_block,end_block,volume
timestamp,,,,,,,,,,,,,,
2020-09-09 19:00:00,NaN,NaN,1098.324341,1027.153442,1098.442139,1024.730103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.334587e+06
2020-09-09 20:00:00,NaN,NaN,1026.515991,986.261597,1029.403931,981.632202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.900282e+06
2020-09-09 21:00:00,NaN,NaN,981.422180,990.119446,1005.434631,976.647522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.269460e+06
2020-09-09 22:00:00,NaN,NaN,989.232422,1005.981018,1013.748352,981.842102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.878710e+06
2020-09-09 23:00:00,NaN,NaN,1007.145752,1005.840698,1028.298218,990.338013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.017900e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-20 04:00:00,NaN,NaN,1955.522827,1967.195190,1976.497192,1942.261597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.922379e+05
2022-08-20 05:00:00,NaN,NaN,1975.574707,1970.000488,1989.517334,1963.135132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.161758e+05
2022-08-20 06:00:00,NaN,NaN,1964.481323,1994.408936,1994.408936,1959.791138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.959464e+05
